# Show how to extract prediction contributions for each distribution parameter

This example shows how to get the contribution of every feature for each distributional parameter for a given data set. This allows similar inferences as one might get from SHAP but comes from lightGBM's internal workings. We can use output for example to get for a given prediction which features are causing the most impact to a given distributional parameter.

These contributions are created before the response function is applied. As such in the case of the identity function, for a given row of data the sum of the contributions should equal the parameter value.


# Imports

First, we import necessary functions. 

In [45]:
import numpy as np

from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from lightgbmlss.datasets.data_loader import load_simulated_gaussian_data
from scipy.stats import norm

import plotnine
from plotnine import *

plotnine.options.figure_size = (12, 8)

# Data

In [46]:
train, test = load_simulated_gaussian_data()

X_train, y_train = train.filter(regex="x"), train["y"].values
X_test, y_test = test.filter(regex="x"), test["y"].values

dtrain = lgb.Dataset(X_train, label=y_train)

# Get a Trained Model

As this example is about th uses of a trained model, we wont do any hyper-parameter searching. We will also use a Gaussian distribution as the response function of the loc parameter is the identity function, this will allow us to more easily compare the outputs of a standard parameter prediction to a contributions prediction.

In [47]:
lgblss = LightGBMLSS(
    Gaussian()
)
lgblss.train(
    params=dict(),
    train_set=dtrain
)

param_dict = {
    "eta":                      ["float", {"low": 1e-5,   "high": 1,     "log": True}],
    "max_depth":                ["int",   {"low": 1,      "high": 10,    "log": False}],
    "num_leaves":               ["int",   {"low": 255,    "high": 255,   "log": False}],  # set to constant for this example
    "min_data_in_leaf":         ["int",   {"low": 20,     "high": 20,    "log": False}],  # set to constant for this example
    "min_gain_to_split":        ["float", {"low": 1e-8,   "high": 40,    "log": False}],
    "min_sum_hessian_in_leaf":  ["float", {"low": 1e-8,   "high": 500,   "log": True}],
    "subsample":                ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "feature_fraction":         ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "boosting":                 ["categorical", ["gbdt"]],
}

np.random.seed(123)
opt_param = lgblss.hyper_opt(param_dict,
                             dtrain,
                             num_boost_round=100,        # Number of boosting iterations.
                             nfold=5,                    # Number of cv-folds.
                             early_stopping_rounds=20,   # Number of early-stopping rounds
                             max_minutes=10,             # Time budget in minutes, i.e., stop study after the given number of minutes.
                             n_trials=30 ,               # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
                             silence=True,               # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
                             seed=123,                   # Seed used to generate cv-folds.
                             hp_seed=123                 # Seed for random number generator used in the Bayesian hyperparameter search.
                            )

np.random.seed(123)

opt_params = opt_param.copy()
n_rounds = opt_params["opt_rounds"]
del opt_params["opt_rounds"]

# Train Model with optimized hyperparameters
lgblss.train(opt_params,
             dtrain,
             num_boost_round=n_rounds
             )


  0%|          | 0/30 [00:00<?, ?it/s]


Hyper-Parameter Optimization successfully finished.
  Number of finished trials:  30
  Best trial:
    Value: 2.0839106241730967
    Params: 
    eta: 0.042322345196562056
    max_depth: 3
    num_leaves: 255
    min_data_in_leaf: 20
    min_gain_to_split: 10.495083287505906
    min_sum_hessian_in_leaf: 4.025662198099785e-06
    subsample: 0.41879883505881144
    feature_fraction: 0.7628021535153005
    boosting: gbdt
    opt_rounds: 72


# Get parameter predictions and parameter contribution predictions

In [48]:
pred_params = lgblss.predict(X_test, pred_type="parameters")
pred_param_contributions = lgblss.predict(X_test, pred_type="contributions")


As location parameter uses identity function the sum of these predictions should equal the value in pred_params. However this is not true for the scale params, as response functions have not been applied when contributions are created.

In [49]:
sum_of_contributions = pred_param_contributions.groupby(level="distribution_arg", axis=1).sum()
location_values_are_all_close =  np.allclose(pred_params["loc"], sum_of_contributions["loc"])
scale_values_are_all_close =  np.allclose(pred_params["scale"], sum_of_contributions["scale"])


print(f"{location_values_are_all_close=}")
print(f"{scale_values_are_all_close=}")


location_values_are_all_close=True
scale_values_are_all_close=False


C:\Users\SimonRobertPike\AppData\Local\Temp\ipykernel_47316\1135199838.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.


### Show contributions for each feature for location parameter

In [50]:
pred_param_contributions.xs("loc", axis=1, level="distribution_arg")

FeatureContribution,x_true,x_noise1,x_noise2,x_noise3,x_noise4,x_noise5,x_noise6,x_noise7,x_noise8,x_noise9,x_noise10,Const
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.979578


### Show contributions for each feature for scale parameter

In [51]:
pred_param_contributions.xs("scale", axis=1, level="distribution_arg")


FeatureContribution,x_true,x_noise1,x_noise2,x_noise3,x_noise4,x_noise5,x_noise6,x_noise7,x_noise8,x_noise9,x_noise10,Const
0,0.410556,0.002107,0.0,0.0,0.000034,0.000197,0.004104,-0.000126,0.0,-0.000608,-0.000503,0.653625
1,0.411267,0.000683,0.0,0.0,-0.000340,0.000197,0.004816,-0.000126,0.0,-0.000608,-0.000130,0.653625
2,-0.597710,0.002107,0.0,0.0,-0.000340,0.000197,0.004104,-0.000126,0.0,-0.000608,-0.000130,0.653625
3,0.848812,0.002835,0.0,0.0,0.000034,0.000197,0.001399,-0.000126,0.0,-0.000608,0.001530,0.653625
4,0.414533,0.001566,0.0,0.0,0.000867,0.000123,0.002717,-0.004173,0.0,0.053938,0.001895,0.653625
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.411235,0.002835,0.0,0.0,-0.000340,0.000197,0.002135,-0.000126,0.0,-0.000608,0.000432,0.653625
2996,0.380668,0.002107,0.0,0.0,-0.000340,0.000197,0.004402,-0.000126,0.0,-0.000608,0.002378,0.653625
2997,-0.597620,0.001648,0.0,0.0,0.000034,0.000197,-0.004548,-0.000126,0.0,-0.000700,-0.000892,0.653625
2998,-0.607374,-0.001427,0.0,0.0,-0.001144,0.000888,0.002017,0.003200,0.0,-0.000029,-0.004399,0.653625


# Show Mean Feature Impact for Data set

In [52]:
sum_of_contributions_column = "SumOfContributions"
mean_parameter_contribution = pred_param_contributions.abs().mean().unstack("distribution_arg")
mean_parameter_contribution[sum_of_contributions_column] = mean_parameter_contribution.sum(1)

mean_parameter_contribution.sort_values(sum_of_contributions_column, ascending=False).drop(columns=sum_of_contributions_column)


distribution_arg,loc,scale
FeatureContribution,,
Const,9.979577,0.653625
x_true,0.000000,0.591884
x_noise6,0.000000,0.004868
x_noise7,0.000000,0.004415
x_noise1,0.000000,0.003994
x_noise10,0.000000,0.002689
x_noise9,0.000000,0.002583
x_noise4,0.000000,0.001668
x_noise5,0.000000,0.000585


# Get correlation between contributions for the scale parameter 

In [53]:
pred_param_contributions.xs("scale", axis=1, level="distribution_arg").corr().dropna(how="all").dropna(axis=1,how="all")


FeatureContribution,x_true,x_noise1,x_noise4,x_noise5,x_noise6,x_noise7,x_noise9,x_noise10
FeatureContribution,,,,,,,,
x_true,1.000000,0.007743,-0.001231,-0.047812,-0.021563,0.015347,0.024365,0.035477
x_noise1,0.007743,1.000000,-0.006635,-0.022209,0.136772,0.002129,-0.006972,0.012110
x_noise4,-0.001231,-0.006635,1.000000,-0.015972,-0.030669,0.474525,0.013082,-0.035711
x_noise5,-0.047812,-0.022209,-0.015972,1.000000,0.006214,0.021844,0.015998,-0.001439
x_noise6,-0.021563,0.136772,-0.030669,0.006214,1.000000,0.029845,0.009551,0.028450
x_noise7,0.015347,0.002129,0.474525,0.021844,0.029845,1.000000,0.023553,-0.015334
x_noise9,0.024365,-0.006972,0.013082,0.015998,0.009551,0.023553,1.000000,-0.030410
x_noise10,0.035477,0.012110,-0.035711,-0.001439,0.028450,-0.015334,-0.030410,1.000000
